# Add Kinetic Inductance Modulation (I* = 1 μA)
Flux-pumped SQUID (L_J) + kinetic inductance L_k(I)=L_{k0}(1+(I/I*)^2).
We compute δn/n and sidebands (dBc) vs I_ac, and a 2D sweep.

In [ ]:
Phi0=2.067833848e-15
I0=3e-06
Istar=1e-06
Lk0=1.5e-10
L_par=2.2e-10
C_line=1.2e-10
Lseg=0.01
f0=6000000000.0
vg_nom=120000000.0


In [ ]:
import numpy as np, matplotlib.pyplot as plt
from mpmath import besselj

def Ic_squid(phi): return 2*I0*abs(np.cos(np.pi*phi/Phi0))

def LJ(phi):
    Ic = Ic_squid(phi)
    Ic = np.maximum(Ic, 1e-12)
    return Phi0/(2*np.pi*Ic)

def Lk(I): return Lk0*(1.0 + (I/Istar)**2)

def delta_n_over_n_flux(phi_dc, phi_ac):
    dphi=1e-18
    L0 = L_par + LJ(phi_dc) + Lk(0.0)
    dLdphi = (LJ(phi_dc + dphi) - LJ(phi_dc - dphi)) / (2*dphi)
    m = (dLdphi / L0) * phi_ac
    return 0.5*m

def delta_n_over_n_kinetic(Idc, Iac):
    dLk_dI = Lk0 * 2.0 * Idc / (Istar**2)
    L0 = L_par + LJ(0.0) + Lk(Idc)
    m = (dLk_dI / L0) * Iac
    return 0.5*m

def beta_from_dnn(dnn, tau=(Lseg/1.2e8)):
    return 2*np.pi*f0 * tau * dnn

def dBc_from_beta(beta):
    J0=float(besselj(0,beta)); J1=float(besselj(1,beta))
    return 10*np.log10(((J1/J0)**2)+1e-30)


In [ ]:
# δn/n vs I_ac for Idc in {0,0.3,0.6,0.9} μA
Idc_vals=[0.0e-6,0.3e-6,0.6e-6,0.9e-6]
Iac=np.linspace(0,1.0e-6,200)
plt.figure(figsize=(7.2,4.4))
for Idc in Idc_vals:
    dnn=[delta_n_over_n_kinetic(Idc,i) for i in Iac]
    plt.plot(Iac*1e6,dnn,label=f"Idc={Idc*1e6:.1f} μA")
plt.xlabel("I_ac (μA)"); plt.ylabel("δn/n (kinetic)"); plt.legend(); plt.title("δn/n vs I_ac (I*=1 μA)"); plt.show()

In [ ]:
# dBc vs I_ac for fixed Φ_dc, Φ_ac; compare flux-only, kinetic-only, combined
import numpy as np
phi_dc=0.30*Phi0; phi_ac=0.08*Phi0; Idc=0.6e-6
Iac=np.linspace(0,1.0e-6,200)
flux=[]; kin=[]; combo=[]
for i in Iac:
    dnn_f=delta_n_over_n_flux(phi_dc,phi_ac); dnn_k=delta_n_over_n_kinetic(Idc,i)
    beta_f=beta_from_dnn(dnn_f); beta_k=beta_from_dnn(dnn_k)
    from mpmath import besselj
    import math
    def dBc(b):
        J0=float(besselj(0,b)); J1=float(besselj(1,b)); return 10*np.log10(((J1/J0)**2)+1e-30)
    flux.append(dBc(beta_f)); kin.append(dBc(beta_k)); combo.append(dBc(beta_f+beta_k))
plt.figure(figsize=(7.2,4.4))
plt.plot(Iac*1e6,flux,label="Flux-only"); plt.plot(Iac*1e6,kin,label="Kinetic-only"); plt.plot(Iac*1e6,combo,label="Combined")
plt.xlabel("I_ac (μA)"); plt.ylabel("P±1/P0 (dBc)"); plt.legend(); plt.title("Sidebands vs I_ac — flux vs kinetic vs combined"); plt.show()

In [ ]:
# Heatmap sweep (Φ_ac/Φ0, I_ac/μA) → dBc
phi_ac_frac=np.linspace(0,0.15,60); Iac_uA=np.linspace(0,1.0,60)
Z=np.zeros((len(Iac_uA),len(phi_ac_frac)))
phi_dc=0.30*Phi0; Idc=0.6e-6
for i,Iu in enumerate(Iac_uA):
    for j,pac in enumerate(phi_ac_frac):
        dnn_f=delta_n_over_n_flux(phi_dc,pac*Phi0)
        dnn_k=delta_n_over_n_kinetic(Idc,Iu*1e-6)
        from mpmath import besselj
        beta=beta_from_dnn(dnn_f+dnn_k)
        J0=float(besselj(0,beta)); J1=float(besselj(1,beta)); ratio=((J1/J0)**2) if J0!=0 else 0.0
        Z[i,j]=10*np.log10(ratio+1e-30)
import matplotlib.pyplot as plt
plt.figure(figsize=(7,4.6))
plt.imshow(Z,origin='lower',aspect='auto',extent=[phi_ac_frac[0],phi_ac_frac[-1],Iac_uA[0],Iac_uA[-1]])
plt.xlabel("Φ_ac/Φ0"); plt.ylabel("I_ac (μA)"); plt.colorbar(label="P±1/P0 (dBc)"); plt.title("Combined dBc heatmap"); plt.show()